# The 20 newsgroups topic analysis

Instead of repeating the IMDB sentiment analysis from the lesson (because frankly, I'm a little bored with sentiment analysis), I will attempt to apply a similar approach to deep-learning NLP classification to a dataset a coworker has recently been messing around with in `scikit-learn`: `sklearn.datasets.fetch_20newsgroups`.

http://people.csail.mit.edu/jrennie/20Newsgroups/

## Setup data

In [1]:
import os
current_dir = os.getcwd()

LESSON_HOME_DIR = current_dir + '/'
DATA_HOME_DIR = LESSON_HOME_DIR + 'data/'

DATASET_DIR = DATA_HOME_DIR + '20_newsgroup/'
MODEL_DIR = DATASET_DIR + 'models/'

In [2]:
if not os.path.exists(MODEL_DIR):
    os.mkdir(DATASET_DIR)
    os.mkdir(MODEL_DIR)

In [3]:
from sklearn.datasets import fetch_20newsgroups

category_subset = [
    'alt.atheism',
    'comp.graphics',
    'comp.os.ms-windows.misc',
    'soc.religion.christian',
]

newsgroups = fetch_20newsgroups(
    subset = 'all',
    categories = category_subset,
    shuffle = True,
    remove = ('headers', 'footers', 'quotes'))

In [4]:
newsgroups.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'soc.religion.christian']

`target_names` are as requested

In [5]:
newsgroups.filenames.shape, newsgroups.target.shape, len(newsgroups.data)

((3754,), (3754,), 3754)

Keras implements `get_word_index()` for the IMDB dataset, which returns an dictionary of word->index derived from a json file hosted on Amazon S3.

It seems bizarre to me to host this when you can easily create it on-demand... anyway, sklearn doesn't provide this. So let's create our own index with `keras.preprocessing.text.Tokenizer` (https://keras.io/preprocessing/text/).

In [6]:
import keras.preprocessing.text
import string

# Workaround to add "Unicode support for keras.preprocessing.text"
# (https://github.com/fchollet/keras/issues/1072#issuecomment-295470970)
def text_to_word_sequence(text,
                          filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                          lower=True, split=" "):
    if lower: text = text.lower()
    if type(text) == unicode:
        translate_table = {ord(c): ord(t) for c,t in zip(filters, split*len(filters)) }
    else:
        translate_table = string.maketrans(filters, split * len(filters))
    text = text.translate(translate_table)
    seq = text.split(split)
    return [i for i in seq if i]
    
keras.preprocessing.text.text_to_word_sequence = text_to_word_sequence

Using Theano backend.
Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [7]:
from keras.preprocessing.text import Tokenizer

vocab_size = 20000

tokenizer = Tokenizer(nb_words=vocab_size)
tokenizer.fit_on_texts(newsgroups.data) # builds the word index
sequences = tokenizer.texts_to_sequences(newsgroups.data)

In [8]:
word_index = tokenizer.word_index

In [9]:
len(word_index)

72905

Reverse the `word_index` with `idx2word`.

In [10]:
idx2word = {v: k for k, v in word_index.iteritems()}

Let's take a look at the first review, both as a list of indices and as text reconstructed from the indices.

In [11]:
', '.join(map(str, sequences[0]))

'24, 2, 60, 566, 52, 20, 4829, 3, 389, 2, 3000, 1339, 2, 155, 386, 84, 901, 76, 4, 115, 24, 2, 88, 566, 76, 92, 402, 3, 525, 12351, 101, 2, 155, 385, 6, 1685, 2454, 236, 93, 182, 118, 1350, 335, 7, 108, 5725, 2764, 6, 3000, 8981, 20, 396, 3, 118, 127, 2454, 6, 158, 182, 90, 4499, 14, 129, 11, 2274, 81, 4808, 234, 219, 92, 23, 10264, 5605, 6, 720, 10, 3000, 61, 783, 5464, 8, 5725, 7, 1736, 3, 239, 3, 1464, 55, 2, 579, 5464, 214, 701, 45, 91, 23, 2, 1802, 4, 11819, 5, 1464, 10, 2, 2694, 74, 318, 10232, 129, 386, 436, 5, 10837, 24, 11, 30, 3, 102, 1641, 11, 114, 332, 8, 2, 1598, 5, 858, 537, 9, 11, 1237, 96, 386, 95, 17, 23, 3, 952, 26, 8, 9, 15584, 121, 23, 219, 9951, 9845, 61, 2104, 95, 96, 5725, 11, 8, 5, 2828, 3, 2501, 197, 4, 127, 863, 720, 200, 102, 337, 127, 1576, 1021, 93, 15, 495, 2, 2157, 4, 94, 396, 3, 5777, 127, 9605, 720, 182, 118, 614, 2221, 60, 261, 3, 583, 8, 9, 135, 2, 154, 1464, 8, 210, 1464'

In [12]:
idx2word[24]

u'on'

In [13]:
' '.join([idx2word[o] for o in sequences[0]])

u"on the one hand there are advantages to having the liturgy stay the same john has described some of these on the other hand some people seem to start tuning out the same old and pay attention better when things get changed around i think innovative priests and liturgy committees are trying to get our attention and make things more meaningful for us it drives me crazy too different people have differing preferences and needs in liturgy my local parish is innovative i prefer to go to mass at the next parish over sometimes we don't have the option of attending a mass in the style which best suits us john put a smiley on it but to just offer it up probably is the solution a related issue that it sounds like john does not have to deal with is that spouses may have different liturgical tastes my husband does like innovative it is a challenge to meet both of our spiritual needs without just going our separate ways when you include the factor of also trying to satisfy our children's needs th

In [14]:
newsgroups.target[0], newsgroups.target_names[newsgroups.target[0]]

(3, 'soc.religion.christian')

Distribution of the lengths of sentences:

In [15]:
import numpy as np

lens = np.array(map(len, newsgroups.data))
(lens.max(), lens.min(), lens.mean())

(158791, 0, 1493.157432072456)

Weird that there are sentences with 0 sequences (words) in them...

In [16]:
# get indices of arrays that do NOT satisfy np.nonzero
nonzero_indices = np.unique(np.nonzero(sequences)[0])
zero_indices = set(range(len(sequences))).difference(nonzero_indices)
len(zero_indices)

101

So there are 101 sentences with no words. E.g.

In [17]:
sequences[64], newsgroups.target_names[newsgroups.target[64]]

([], 'alt.atheism')

...sure.

Pad (with zero) or truncate each sentence to make consistent length.

In [18]:
from keras.preprocessing import sequence

seq_len = 1000

data = sequence.pad_sequences(sequences, maxlen=seq_len, value=0)

In [19]:
data[:10]

array([[    0,     0,     0, ...,     8,   210,  1464],
       [    0,     0,     0, ...,  3162,     8,    11],
       [    0,     0,     0, ...,     2,   318,  1142],
       ..., 
       [    0,     0,     0, ...,    47,     7,   740],
       [ 1070,    11,     8, ...,  2565,   356,   129],
       [    0,     0,     0, ..., 16529,   364,  8254]], dtype=int32)

Finally, let's turn the labels into categorical information.

In [20]:
from keras.utils.np_utils import to_categorical

newsgroups.target = to_categorical(np.asarray(newsgroups.target))

In [21]:
data.shape, newsgroups.target.shape

((3754, 1000), (3754, 4))

Split data into train-test.

In [22]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data, newsgroups.target, test_size=0.33)

In [23]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((2515, 1000), (1239, 1000), (2515, 4), (1239, 4))

## Create simple models

### Single hidden layer NN

The simplest model that tends to give reasonable results is a single hidden layer net. So let's try that. Note that we can't expect to get any useful results by feeding word ids directly into a neural net - so instead we use an embedding to replace them with a vector of 32 (initially random) floats for each word in the vocab.

In [24]:
vocab_size, seq_len

(20000, 1000)

In [25]:
from keras.models import Sequential
from keras.layers import Embedding
from keras.layers.core import Flatten, Dense, Dropout
from keras.optimizers import Adam

# input_length => 1500-word reviews, 32 floats per word
model = Sequential([
    Embedding(vocab_size, 32, input_length=seq_len),
    Flatten(),
    Dense(100, activation='relu'),
    Dropout(0.7),
    Dense(len(newsgroups.target_names), activation='softmax')])

In [26]:
model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_1 (Embedding)          (None, 1000, 32)      640000      embedding_input_1[0][0]          
____________________________________________________________________________________________________
flatten_1 (Flatten)              (None, 32000)         0           embedding_1[0][0]                
____________________________________________________________________________________________________
dense_1 (Dense)                  (None, 100)           3200100     flatten_1[0][0]                  
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 100)           0           dense_1[0][0]                    
___________________________________________________________________________________________

In [27]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=10, batch_size=64)

Train on 2515 samples, validate on 1239 samples
Epoch 1/10
2515/2515 [==============================] - 0s - loss: 1.4164 - acc: 0.2958 - val_loss: 1.3311 - val_acc: 0.3858
Epoch 2/10
2515/2515 [==============================] - 0s - loss: 1.2913 - acc: 0.3658 - val_loss: 1.2087 - val_acc: 0.4487
Epoch 3/10
2515/2515 [==============================] - 0s - loss: 1.0958 - acc: 0.4680 - val_loss: 0.9064 - val_acc: 0.5908
Epoch 4/10
2515/2515 [==============================] - 0s - loss: 0.7658 - acc: 0.6533 - val_loss: 0.7439 - val_acc: 0.6312
Epoch 5/10
2515/2515 [==============================] - 0s - loss: 0.4885 - acc: 0.8278 - val_loss: 0.6599 - val_acc: 0.6941
Epoch 6/10
2515/2515 [==============================] - 0s - loss: 0.2883 - acc: 0.9205 - val_loss: 0.5805 - val_acc: 0.7530
Epoch 7/10
2515/2515 [==============================] - 0s - loss: 0.1777 - acc: 0.9610 - val_loss: 0.5618 - val_acc: 0.7554
Epoch 8/10
2515/2515 [==============================] - 0s - loss: 0.1319 - a

Is a mid-70s validation accuracy.. Good? Bad?

Here are some accuracies [from an official `sklearn` example](http://scikit-learn.org/stable/auto_examples/text/document_classification_20newsgroups.html) that classifies documents by topics using a bag-of-words approach:

```
[('RidgeClassifier', 0.89726533628972649),
 ('Perceptron', 0.88543976348854403),
 ('PassiveAggressiveClassifier', 0.90613451589061345),
 ('KNeighborsClassifier', 0.85809312638580926),
 ('RandomForestClassifier', 0.83813747228381374),
 ('LinearSVC', 0.90022172949002222),
 ('SGDClassifier', 0.90096082779009612),
 ('LinearSVC', 0.87287509238728755),
 ('SGDClassifier', 0.88543976348854403),
 ('SGDClassifier', 0.89874353288987441),
 ('NearestCentroid', 0.85513673318551364),
 ('MultinomialNB', 0.90022172949002222),
 ('BernoulliNB', 0.88396156688839611),
 ('Pipeline', 0.8810051736881005)]
 
 mean: 0.88311688311688319
 ```

So, not a good result in comparison with much simpler approaches. Training accuracy is high, but testing accuracy is much poorer.

As a sanity check, I also ran code from [`pretrained_word_embeddings.py`](https://github.com/fchollet/keras/blob/master/examples/pretrained_word_embeddings.py) (from Keras's examples repository) which also runs against `20_newsgroups` (not the `sklearn` version though), and it was able to achieve:

    loss: 0.3784 - acc: 0.8734 - val_loss: 0.9177 - val_acc: 0.7257
after 10 epochs - again, not as accurate as the 'shallow', bag-of-words models - but comparable to the results I'm receiving here.

### Single conv layer with max pooling

A CNN is likely to work better, since it's designed to take advantage of ordered data. We'll need to use a 1D CNN, since a sequence of words is 1D.

In [28]:
from keras.layers.convolutional import Convolution1D, MaxPooling1D

conv1 = Sequential([
    Embedding(vocab_size, 100, input_length=seq_len, dropout=0.2),
    Dropout(0.4),
    Convolution1D(128, 5, activation='relu'),
    Dropout(0.4),
    MaxPooling1D(5),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.7),
    Dense(len(newsgroups.target_names), activation='softmax')])

In [29]:
from keras.optimizers import RMSprop

conv1.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])
conv1.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_2 (Embedding)          (None, 1000, 100)     2000000     embedding_input_2[0][0]          
____________________________________________________________________________________________________
dropout_2 (Dropout)              (None, 1000, 100)     0           embedding_2[0][0]                
____________________________________________________________________________________________________
convolution1d_1 (Convolution1D)  (None, 996, 128)      64128       dropout_2[0][0]                  
____________________________________________________________________________________________________
dropout_3 (Dropout)              (None, 996, 128)      0           convolution1d_1[0][0]            
___________________________________________________________________________________________

In [30]:
conv1.optimizer.lr.get_value().item()

0.0010000000474974513

In [31]:
conv1.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=10, batch_size=64)

Train on 2515 samples, validate on 1239 samples
Epoch 1/10
2515/2515 [==============================] - 4s - loss: 1.4193 - acc: 0.3026 - val_loss: 1.3534 - val_acc: 0.3721
Epoch 2/10
2515/2515 [==============================] - 4s - loss: 1.3302 - acc: 0.3475 - val_loss: 1.2985 - val_acc: 0.4116
Epoch 3/10
2515/2515 [==============================] - 4s - loss: 1.1868 - acc: 0.4223 - val_loss: 1.0413 - val_acc: 0.5020
Epoch 4/10
2515/2515 [==============================] - 4s - loss: 0.9257 - acc: 0.5169 - val_loss: 0.8350 - val_acc: 0.5650
Epoch 5/10
2515/2515 [==============================] - 4s - loss: 0.7874 - acc: 0.5773 - val_loss: 0.7488 - val_acc: 0.6481
Epoch 6/10
2515/2515 [==============================] - 4s - loss: 0.6623 - acc: 0.6835 - val_loss: 0.6472 - val_acc: 0.7490
Epoch 7/10
2515/2515 [==============================] - 4s - loss: 0.5284 - acc: 0.7809 - val_loss: 0.5500 - val_acc: 0.7756
Epoch 8/10
2515/2515 [==============================] - 4s - loss: 0.4189 - a

In [32]:
conv1.optimizer.lr=0.01

In [33]:
conv1.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=4, batch_size=64)

Train on 2515 samples, validate on 1239 samples
Epoch 1/4
2515/2515 [==============================] - 4s - loss: 0.2564 - acc: 0.9054 - val_loss: 0.5452 - val_acc: 0.7837
Epoch 2/4
2515/2515 [==============================] - 4s - loss: 0.2232 - acc: 0.9137 - val_loss: 0.5260 - val_acc: 0.8015
Epoch 3/4
2515/2515 [==============================] - 4s - loss: 0.2021 - acc: 0.9193 - val_loss: 0.5464 - val_acc: 0.7910
Epoch 4/4
2515/2515 [==============================] - 4s - loss: 0.1948 - acc: 0.9221 - val_loss: 0.6848 - val_acc: 0.7724


In [34]:
conv1.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=1, batch_size=64)

Train on 2515 samples, validate on 1239 samples
Epoch 1/1
2515/2515 [==============================] - 4s - loss: 0.1794 - acc: 0.9344 - val_loss: 0.5481 - val_acc: 0.7990


A good improvement over the previous model.

## Pre-trained vectors

You may want to look at wordvectors.ipynb before moving on.

In this section, we replicate the previous CNN, but using pre-trained embeddings.

In [35]:
from keras.utils.data_utils import get_file

def get_glove_dataset(dataset):
    """Download the requested glove dataset from files.fast.ai
    and return a location that can be passed to load_vectors.
    """
    # see wordvectors.ipynb for info on how these files were
    # generated from the original glove data.
    md5sums = {'6B.50d': '8e1557d1228decbda7db6dfd81cd9909',
               '6B.100d': 'c92dbbeacde2b0384a43014885a60b2c',
               '6B.200d': 'af271b46c04b0b2e41a84d8cd806178d',
               '6B.300d': '30290210376887dcc6d0a5a6374d8255'}
    glove_path = os.path.abspath('data/glove/results')
    %mkdir -p $glove_path
    return get_file(dataset,
                    'http://files.fast.ai/models/glove/' + dataset + '.tgz',
                    cache_subdir=glove_path,
                    md5_hash=md5sums.get(dataset, None),
                    untar=True)

In [36]:
from utils import load_array
import pickle

def load_vectors(loc):
    return (load_array(loc+'.dat'),
        pickle.load(open(loc+'_words.pkl','rb')),
        pickle.load(open(loc+'_idx.pkl','rb')))

In [37]:
vecs, words, wordidx = load_vectors(get_glove_dataset('6B.100d'))

Untaring file...


In [38]:
len(wordidx)

400000

The glove word ids and imdb word ids use different indexes. So we create a simple function that creates an embedding matrix using the indexes from imdb, and the embeddings from glove (where they exist).

In [39]:
import re
from numpy.random import normal

def create_emb():
    n_fact = vecs.shape[1]
    emb = np.zeros((vocab_size, n_fact))

    for i in range(1,len(emb)):
        word = idx2word[i]
        if word and word in wordidx:
            src_idx = wordidx[word]
            emb[i] = vecs[src_idx]
        else:
            # If we can't find the word in glove, randomly initialize
            emb[i] = normal(scale=0.6, size=(n_fact,))

    # This is our "rare word" id - we want to randomly initialize
    emb[-1] = normal(scale=0.6, size=(n_fact,))
    emb/=3
    return emb

In [40]:
emb = create_emb()

In [41]:
emb.shape

(20000, 100)

In [42]:
emb_model = Sequential([
    Embedding(vocab_size, 100, input_length=seq_len, dropout=0.2, 
              weights=[emb], trainable=False),
    Dropout(0.25),
    Convolution1D(128, 5, border_mode='same', activation='relu'),
    Dropout(0.25),
    MaxPooling1D(),
    Flatten(),
    Dense(100, activation='relu'),
    Dropout(0.7),
    Dense(4, activation='softmax')])

_Note_: I started seeing lines like `4s - loss: nan - acc: 0.6783 - val_loss: nan - val_acc: 0.2131` where in the previous epoch, `val_acc` was twice that amount. A [quick search on the forums](http://forums.fast.ai/t/why-are-my-losses-nan/2931/2) surfaced this explanation:

    "There is one thing that doesn't look quite right: the final activation is not compatible with that loss function. Categorical cross-entropy expects a 'softmax' activation in the final layer, not 'sigmoid'. Consider changing that to see what happens."
    
**Categorical cross-entropy expects a `softmax` activation in the final layer, not `sigmoid`.** So I switched to `softmax`... I don't recall ever learning this information, however. Should ponder why.

In [43]:
emb_model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])
emb_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_3 (Embedding)          (None, 1000, 100)     0           embedding_input_3[0][0]          
____________________________________________________________________________________________________
dropout_5 (Dropout)              (None, 1000, 100)     0           embedding_3[0][0]                
____________________________________________________________________________________________________
convolution1d_2 (Convolution1D)  (None, 1000, 128)     64128       dropout_5[0][0]                  
____________________________________________________________________________________________________
dropout_6 (Dropout)              (None, 1000, 128)     0           convolution1d_2[0][0]            
___________________________________________________________________________________________

In [44]:
emb_model.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=10, batch_size=64)

Train on 2515 samples, validate on 1239 samples
Epoch 1/10
2515/2515 [==============================] - 4s - loss: 1.4046 - acc: 0.3324 - val_loss: 1.2151 - val_acc: 0.4746
Epoch 2/10
2515/2515 [==============================] - 4s - loss: 1.1134 - acc: 0.4608 - val_loss: 0.9404 - val_acc: 0.5174
Epoch 3/10
2515/2515 [==============================] - 4s - loss: 0.9463 - acc: 0.5082 - val_loss: 0.8698 - val_acc: 0.5456
Epoch 4/10
2515/2515 [==============================] - 4s - loss: 0.8846 - acc: 0.5348 - val_loss: 0.8030 - val_acc: 0.5811
Epoch 5/10
2515/2515 [==============================] - 4s - loss: 0.8320 - acc: 0.5750 - val_loss: 0.7712 - val_acc: 0.6465
Epoch 6/10
2515/2515 [==============================] - 4s - loss: 0.7719 - acc: 0.6203 - val_loss: 0.7430 - val_acc: 0.6772
Epoch 7/10
2515/2515 [==============================] - 4s - loss: 0.7437 - acc: 0.6473 - val_loss: 0.7118 - val_acc: 0.7022
Epoch 8/10
2515/2515 [==============================] - 4s - loss: 0.7048 - a

Let's fine-tune the embedding weights - especially since the words we couldn't find in glove just have random embeddings.

In [45]:
emb_model.layers[0].trainable=True

In [46]:
emb_model.optimizer.lr=1e-4

In [47]:
emb_model.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=10, batch_size=64)

Train on 2515 samples, validate on 1239 samples
Epoch 1/10
2515/2515 [==============================] - 4s - loss: 0.6127 - acc: 0.7272 - val_loss: 0.6395 - val_acc: 0.7191
Epoch 2/10
2515/2515 [==============================] - 4s - loss: 0.5789 - acc: 0.7539 - val_loss: 0.6408 - val_acc: 0.7272
Epoch 3/10
2515/2515 [==============================] - 4s - loss: 0.5846 - acc: 0.7543 - val_loss: 0.6229 - val_acc: 0.7393
Epoch 4/10
2515/2515 [==============================] - 4s - loss: 0.5296 - acc: 0.7706 - val_loss: 0.6232 - val_acc: 0.7337
Epoch 5/10
2515/2515 [==============================] - 4s - loss: 0.5280 - acc: 0.7734 - val_loss: 0.6175 - val_acc: 0.7441
Epoch 6/10
2515/2515 [==============================] - 4s - loss: 0.5127 - acc: 0.7873 - val_loss: 0.6373 - val_acc: 0.7143
Epoch 7/10
2515/2515 [==============================] - 4s - loss: 0.4915 - acc: 0.7960 - val_loss: 0.6338 - val_acc: 0.7320
Epoch 8/10
2515/2515 [==============================] - 4s - loss: 0.4585 - a

In [48]:
emb_model.layers[0].trainable=False

In [49]:
emb_model.optimizer.lr=1e-2

In [50]:
emb_model.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=5, batch_size=64)

Train on 2515 samples, validate on 1239 samples
Epoch 1/5
2515/2515 [==============================] - 4s - loss: 0.3904 - acc: 0.8461 - val_loss: 0.6197 - val_acc: 0.7401
Epoch 2/5
2515/2515 [==============================] - 4s - loss: 0.3812 - acc: 0.8394 - val_loss: 0.6120 - val_acc: 0.7514
Epoch 3/5
2515/2515 [==============================] - 4s - loss: 0.3672 - acc: 0.8600 - val_loss: 0.6053 - val_acc: 0.7571
Epoch 4/5
2515/2515 [==============================] - 4s - loss: 0.3674 - acc: 0.8612 - val_loss: 0.6129 - val_acc: 0.7506
Epoch 5/5
2515/2515 [==============================] - 4s - loss: 0.3627 - acc: 0.8473 - val_loss: 0.6123 - val_acc: 0.7482


Interestingly, the pretrained embeddings didn't provide any improvement...

## Pre-trained vectors + BatchNorm

In [51]:
emb = create_emb()

In [52]:
from keras.layers.normalization import BatchNormalization

batch_model = Sequential([
    Embedding(vocab_size, 100, input_length=seq_len, dropout=0.2, 
              weights=[emb], trainable=False),
    Dropout(0.25),
    Convolution1D(128, 5, border_mode='same', activation='relu'),
    Dropout(0.25),
    MaxPooling1D(),
    Flatten(),
    Dense(100, activation='relu'),
    BatchNormalization(axis=1),
    Dropout(0.7),
    Dense(4, activation='softmax')])

In [53]:
batch_model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])
batch_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_4 (Embedding)          (None, 1000, 100)     0           embedding_input_4[0][0]          
____________________________________________________________________________________________________
dropout_8 (Dropout)              (None, 1000, 100)     0           embedding_4[0][0]                
____________________________________________________________________________________________________
convolution1d_3 (Convolution1D)  (None, 1000, 128)     64128       dropout_8[0][0]                  
____________________________________________________________________________________________________
dropout_9 (Dropout)              (None, 1000, 128)     0           convolution1d_3[0][0]            
___________________________________________________________________________________________

In [54]:
batch_model.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=10, batch_size=64)

Train on 2515 samples, validate on 1239 samples
Epoch 1/10
2515/2515 [==============================] - 4s - loss: 2.0294 - acc: 0.3006 - val_loss: 1.3218 - val_acc: 0.3527
Epoch 2/10
2515/2515 [==============================] - 4s - loss: 1.4037 - acc: 0.4195 - val_loss: 1.0957 - val_acc: 0.5182
Epoch 3/10
2515/2515 [==============================] - 4s - loss: 1.1374 - acc: 0.5074 - val_loss: 1.0403 - val_acc: 0.4939
Epoch 4/10
2515/2515 [==============================] - 4s - loss: 0.9541 - acc: 0.5539 - val_loss: 0.9714 - val_acc: 0.5779
Epoch 5/10
2515/2515 [==============================] - 4s - loss: 0.8759 - acc: 0.6016 - val_loss: 0.9225 - val_acc: 0.6134
Epoch 6/10
2515/2515 [==============================] - 4s - loss: 0.7938 - acc: 0.6505 - val_loss: 0.9171 - val_acc: 0.5755
Epoch 7/10
2515/2515 [==============================] - 4s - loss: 0.7493 - acc: 0.6740 - val_loss: 0.8804 - val_acc: 0.6489
Epoch 8/10
2515/2515 [==============================] - 4s - loss: 0.7249 - a

In [55]:
batch_model.layers[0].trainable=True

In [56]:
batch_model.optimizer.lr=1e-4

In [57]:
batch_model.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=10, batch_size=64)

Train on 2515 samples, validate on 1239 samples
Epoch 1/10
2515/2515 [==============================] - 4s - loss: 0.5675 - acc: 0.7698 - val_loss: 0.7829 - val_acc: 0.6529
Epoch 2/10
2515/2515 [==============================] - 4s - loss: 0.5324 - acc: 0.7865 - val_loss: 0.7195 - val_acc: 0.7038
Epoch 3/10
2515/2515 [==============================] - 4s - loss: 0.5140 - acc: 0.8072 - val_loss: 0.7830 - val_acc: 0.6295
Epoch 4/10
2515/2515 [==============================] - 4s - loss: 0.5166 - acc: 0.7936 - val_loss: 0.6778 - val_acc: 0.6965
Epoch 5/10
2515/2515 [==============================] - 4s - loss: 0.4869 - acc: 0.8123 - val_loss: 0.6580 - val_acc: 0.7135
Epoch 6/10
2515/2515 [==============================] - 4s - loss: 0.4773 - acc: 0.8159 - val_loss: 0.6609 - val_acc: 0.7159
Epoch 7/10
2515/2515 [==============================] - 4s - loss: 0.4712 - acc: 0.8147 - val_loss: 0.6684 - val_acc: 0.7119
Epoch 8/10
2515/2515 [==============================] - 4s - loss: 0.4312 - a

In [58]:
batch_model.layers[0].trainable=False

In [59]:
batch_model.optimizer.lr=1e-2

In [60]:
batch_model.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=5, batch_size=64)

Train on 2515 samples, validate on 1239 samples
Epoch 1/5
2515/2515 [==============================] - 4s - loss: 0.3744 - acc: 0.8696 - val_loss: 0.6855 - val_acc: 0.7070
Epoch 2/5
2515/2515 [==============================] - 4s - loss: 0.3469 - acc: 0.8716 - val_loss: 0.6401 - val_acc: 0.7296
Epoch 3/5
2515/2515 [==============================] - 4s - loss: 0.3322 - acc: 0.8835 - val_loss: 0.6471 - val_acc: 0.7167
Epoch 4/5
2515/2515 [==============================] - 4s - loss: 0.3293 - acc: 0.8839 - val_loss: 0.7200 - val_acc: 0.6925
Epoch 5/5
2515/2515 [==============================] - 4s - loss: 0.3265 - acc: 0.8783 - val_loss: 0.7172 - val_acc: 0.7022


Nada.

## Pre-trained vectors III

Let's try the model from `pretrained_word_embeddings.py (+Dropout)`

In [61]:
emb = create_emb()

In [62]:
deep_model = Sequential([
    Embedding(vocab_size, 100, input_length=seq_len, weights=[emb], trainable=False),
    Dropout(0.25),
    Convolution1D(128, 5, activation='relu'),
    Dropout(0.25),
    MaxPooling1D(5),
    Convolution1D(128, 5, activation='relu'),
    Dropout(0.25),
    MaxPooling1D(5),
    Convolution1D(128, 5, activation='relu'),
    Dropout(0.25),
    MaxPooling1D(35), # global max pooling
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(newsgroups.target_names), activation='softmax')
])

In [63]:
deep_model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])
deep_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_5 (Embedding)          (None, 1000, 100)     0           embedding_input_5[0][0]          
____________________________________________________________________________________________________
dropout_11 (Dropout)             (None, 1000, 100)     0           embedding_5[0][0]                
____________________________________________________________________________________________________
convolution1d_4 (Convolution1D)  (None, 996, 128)      64128       dropout_11[0][0]                 
____________________________________________________________________________________________________
dropout_12 (Dropout)             (None, 996, 128)      0           convolution1d_4[0][0]            
___________________________________________________________________________________________

In [64]:
deep_model.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=10, batch_size=128)

Train on 2515 samples, validate on 1239 samples
Epoch 1/10
2515/2515 [==============================] - 5s - loss: 1.3067 - acc: 0.3626 - val_loss: 1.1742 - val_acc: 0.4915
Epoch 2/10
2515/2515 [==============================] - 4s - loss: 0.9799 - acc: 0.4831 - val_loss: 0.9304 - val_acc: 0.5174
Epoch 3/10
2515/2515 [==============================] - 4s - loss: 0.8901 - acc: 0.5034 - val_loss: 0.8854 - val_acc: 0.5408
Epoch 4/10
2515/2515 [==============================] - 4s - loss: 0.8379 - acc: 0.5252 - val_loss: 0.8380 - val_acc: 0.5634
Epoch 5/10
2515/2515 [==============================] - 4s - loss: 0.7984 - acc: 0.5519 - val_loss: 0.8056 - val_acc: 0.5932
Epoch 6/10
2515/2515 [==============================] - 4s - loss: 0.7781 - acc: 0.5797 - val_loss: 0.7802 - val_acc: 0.6182
Epoch 7/10
2515/2515 [==============================] - 4s - loss: 0.7103 - acc: 0.6485 - val_loss: 0.7086 - val_acc: 0.7062
Epoch 8/10
2515/2515 [==============================] - 4s - loss: 0.6369 - a

In [65]:
deep_model.layers[0].trainable=True

In [66]:
deep_model.optimizer.lr=1e-4

In [67]:
deep_model.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=10, batch_size=128)

Train on 2515 samples, validate on 1239 samples
Epoch 1/10
2515/2515 [==============================] - 4s - loss: 0.5348 - acc: 0.7682 - val_loss: 0.5906 - val_acc: 0.7425
Epoch 2/10
2515/2515 [==============================] - 4s - loss: 0.4975 - acc: 0.7909 - val_loss: 0.5822 - val_acc: 0.7409
Epoch 3/10
2515/2515 [==============================] - 4s - loss: 0.4793 - acc: 0.8060 - val_loss: 0.5587 - val_acc: 0.7635
Epoch 4/10
2515/2515 [==============================] - 4s - loss: 0.4585 - acc: 0.8068 - val_loss: 0.5478 - val_acc: 0.7676
Epoch 5/10
2515/2515 [==============================] - 4s - loss: 0.4205 - acc: 0.8286 - val_loss: 0.5299 - val_acc: 0.7627
Epoch 6/10
2515/2515 [==============================] - 4s - loss: 0.4099 - acc: 0.8433 - val_loss: 0.5428 - val_acc: 0.7716
Epoch 7/10
2515/2515 [==============================] - 4s - loss: 0.3772 - acc: 0.8453 - val_loss: 0.5232 - val_acc: 0.7780
Epoch 8/10
2515/2515 [==============================] - 4s - loss: 0.3518 - a

In [68]:
deep_model.optimizer.lr=1e-3

In [69]:
deep_model.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=10, batch_size=128)

Train on 2515 samples, validate on 1239 samples
Epoch 1/10
2515/2515 [==============================] - 4s - loss: 0.2940 - acc: 0.8791 - val_loss: 0.5098 - val_acc: 0.7861
Epoch 2/10
2515/2515 [==============================] - 4s - loss: 0.2796 - acc: 0.8891 - val_loss: 0.5084 - val_acc: 0.7764
Epoch 3/10
2515/2515 [==============================] - 4s - loss: 0.2991 - acc: 0.8807 - val_loss: 0.5138 - val_acc: 0.7829
Epoch 4/10
2515/2515 [==============================] - 4s - loss: 0.2819 - acc: 0.8819 - val_loss: 0.5162 - val_acc: 0.7732
Epoch 5/10
2515/2515 [==============================] - 4s - loss: 0.2503 - acc: 0.9010 - val_loss: 0.4986 - val_acc: 0.7950
Epoch 6/10
2515/2515 [==============================] - 4s - loss: 0.2652 - acc: 0.8871 - val_loss: 0.5539 - val_acc: 0.7748
Epoch 7/10
2515/2515 [==============================] - 4s - loss: 0.2645 - acc: 0.8954 - val_loss: 0.4971 - val_acc: 0.7926
Epoch 8/10
2515/2515 [==============================] - 4s - loss: 0.2449 - a

Better than the first two pretrained embedding models and back to being competitive with the single conv layer with max pooling.

## Multi-size CNN

This is an implementation of a multi-size CNN as shown in Ben Bowles' [excellent blog post](https://quid.com/feed/how-quid-uses-deep-learning-with-small-data).

In [70]:
from keras.layers import Merge

We use the functional API to create multiple conv layers of different sizes, and then concatenate them.

In [71]:
from keras.layers import Input, Merge
from keras.models import Model

graph_in = Input((vocab_size, 100))
convs = [] 
for fsz in range (3, 6): 
    x = Convolution1D(64, fsz, border_mode='same', activation='relu')(graph_in)
    x = MaxPooling1D()(x) 
    x = Flatten()(x) 
    convs.append(x)
out = Merge(mode='concat')(convs) 
graph = Model(graph_in, out) 

In [72]:
emb = create_emb()

We then replace the conv/max-pool layer in our original CNN with the concatenated conv layers.

In [73]:
multi = Sequential ([
    Embedding(vocab_size, 100, input_length=seq_len, dropout=0.2, weights=[emb]),
    Dropout (0.2),
    graph,
    Dropout (0.5),
    Dense (100, activation='relu'),
    Dropout (0.7),
    Dense (len(newsgroups.target_names), activation='softmax')
    ])

In [74]:
multi.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])
multi.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_6 (Embedding)          (None, 1000, 100)     2000000     embedding_input_6[0][0]          
____________________________________________________________________________________________________
dropout_16 (Dropout)             (None, 1000, 100)     0           embedding_6[0][0]                
____________________________________________________________________________________________________
model_1 (Model)                  multiple              76992       dropout_16[0][0]                 
____________________________________________________________________________________________________
dropout_17 (Dropout)             (None, 96000)         0           model_1[1][0]                    
___________________________________________________________________________________________

In [75]:
multi.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=10, batch_size=128)

Train on 2515 samples, validate on 1239 samples
Epoch 1/10
2515/2515 [==============================] - 7s - loss: 1.4489 - acc: 0.3233 - val_loss: 1.3003 - val_acc: 0.4705
Epoch 2/10
2515/2515 [==============================] - 7s - loss: 1.1607 - acc: 0.4517 - val_loss: 0.9248 - val_acc: 0.5400
Epoch 3/10
2515/2515 [==============================] - 7s - loss: 0.9511 - acc: 0.5093 - val_loss: 0.8174 - val_acc: 0.5569
Epoch 4/10
2515/2515 [==============================] - 7s - loss: 0.8606 - acc: 0.5519 - val_loss: 0.7706 - val_acc: 0.5763
Epoch 5/10
2515/2515 [==============================] - 7s - loss: 0.7963 - acc: 0.5940 - val_loss: 0.7573 - val_acc: 0.5851
Epoch 6/10
2515/2515 [==============================] - 7s - loss: 0.7287 - acc: 0.6262 - val_loss: 0.7073 - val_acc: 0.6731
Epoch 7/10
2515/2515 [==============================] - 7s - loss: 0.6430 - acc: 0.7042 - val_loss: 0.6469 - val_acc: 0.6893
Epoch 8/10
2515/2515 [==============================] - 7s - loss: 0.5849 - a

In [76]:
multi.layers[0].trainable=False

In [77]:
multi.optimizer.lr=1e-5

In [78]:
multi.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=10, batch_size=128)

Train on 2515 samples, validate on 1239 samples
Epoch 1/10
2515/2515 [==============================] - 7s - loss: 0.3755 - acc: 0.8573 - val_loss: 0.5221 - val_acc: 0.7845
Epoch 2/10
2515/2515 [==============================] - 7s - loss: 0.3282 - acc: 0.8720 - val_loss: 0.5383 - val_acc: 0.7845
Epoch 3/10
2515/2515 [==============================] - 7s - loss: 0.2715 - acc: 0.8970 - val_loss: 0.5014 - val_acc: 0.7950
Epoch 4/10
2515/2515 [==============================] - 7s - loss: 0.2517 - acc: 0.9014 - val_loss: 0.5058 - val_acc: 0.7998
Epoch 5/10
2515/2515 [==============================] - 7s - loss: 0.2158 - acc: 0.9161 - val_loss: 0.5131 - val_acc: 0.8111
Epoch 6/10
2515/2515 [==============================] - 7s - loss: 0.1933 - acc: 0.9272 - val_loss: 0.5480 - val_acc: 0.8015
Epoch 7/10
2515/2515 [==============================] - 7s - loss: 0.1809 - acc: 0.9352 - val_loss: 0.5934 - val_acc: 0.7934
Epoch 8/10
2515/2515 [==============================] - 7s - loss: 0.1690 - a

In [79]:
multi.layers[0].trainable=True

In [80]:
multi.optimizer.lr=1e-2

In [81]:
multi.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=5, batch_size=128)

Train on 2515 samples, validate on 1239 samples
Epoch 1/5
2515/2515 [==============================] - 7s - loss: 0.1322 - acc: 0.9531 - val_loss: 0.6454 - val_acc: 0.7982
Epoch 2/5
2515/2515 [==============================] - 7s - loss: 0.1354 - acc: 0.9523 - val_loss: 0.6256 - val_acc: 0.8031
Epoch 3/5
2515/2515 [==============================] - 7s - loss: 0.1275 - acc: 0.9571 - val_loss: 0.5821 - val_acc: 0.8119
Epoch 4/5
2515/2515 [==============================] - 7s - loss: 0.1091 - acc: 0.9614 - val_loss: 0.5987 - val_acc: 0.8103
Epoch 5/5
2515/2515 [==============================] - 7s - loss: 0.1094 - acc: 0.9586 - val_loss: 0.5981 - val_acc: 0.8095


Highest deep-learning result so far! And the most comparable to the 'shallow' bag-of-words results that achieved upper-80s.

Bonus: this was also the least stressful to watch train because `val_acc` (generally) continued to rise instead of bouncing around like the other models.

## lda2vec

http://nbviewer.jupyter.org/github/cemoody/lda2vec/blob/master/examples/twenty_newsgroups/lda2vec/lda2vec.ipynb

http://lda2vec.readthedocs.io/en/latest/

### Install `lda2vec` and accompanying Python module

1. Download `setup.py`, `requirements.txt`, and the `lda2vec/` folder from https://github.com/cemoody/lda2vec (I used `wget`)
2. Run `python setup.py install`
3. To avoid [this error](https://github.com/explosion/spaCy/issues/855), re-installed Spacy with `conda install spacy -c conda-forge`
4. To [enable CUDA support for `chainer`](https://github.com/chainer/chainer#installation), run `pip install cupy`

Also: `conda install seaborn` for later

### Generate preprocessed data files

_Because `lda2vec` is not fully polished, there are a few steps beyond just running `preprocess.py`_

Download `GoogleNews-vectors-negative300.bin.gz` (1.53GB) and unzip it:

    wget -c https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
    gzip -d GoogleNews-vectors-negative300.bin.gz

Run `pip install pyxdameraulevenshtein` (to get access to `damerau_levenshtein_distance_ndarray`\* in `corpus.py`)

\* Had to replace `damerau_levenshtein_distance_withNPArray`

Install `gensim` (we'll be using the `gensim.models.KeyedVectors` module)\*.

\* Note that the `lda2vec` module's [`corpus.py`](https://github.com/cemoody/lda2vec/blob/master/lda2vec/corpus.py) suggests it provides an option to `use_spacy` to load in word vectors, but this appears broken. Only option is `gensim` at the moment. `corpus.py` must also be modified from using the `gensim.models.word2vec` import to `gensim.models.KeyedVectors`

Finally, run either the code block below, or [preprocess.py](https://raw.githubusercontent.com/cemoody/lda2vec/master/examples/twenty_newsgroups/data/preprocess.py) in `examples/twenty_newsgroups/data` to generate the `20_newsgroup` data that `lda2vec` will run on.

In [ ]:
#!python data/preprocess.py
# from https://raw.githubusercontent.com/cemoody/lda2vec/master/examples/twenty_newsgroups/data/preprocess.py

# Author: Chris Moody <chrisemoody@gmail.com>
# License: MIT

# This simple example loads the newsgroups data from sklearn
# and train an LDA-like model on it
import pickle

from sklearn.datasets import fetch_20newsgroups
import numpy as np

from lda2vec import preprocess, Corpus

# Fetch data
remove = ('headers', 'footers', 'quotes')
category_subset = [
    'alt.atheism',
    'comp.graphics',
    'comp.os.ms-windows.misc',
    'soc.religion.christian',
]
texts = fetch_20newsgroups(subset='all', categories=category_subset, remove=remove).data
# Remove tokens with these substrings
bad = set(["ax>", '`@("', '---', '===', '^^^'])

def clean(line):
    return ' '.join(w for w in line.split() if not any(t in w for t in bad))

# Preprocess data
max_length = 10000   # Limit of 10k words per document
# Convert to unicode (spaCy only works with unicode)
texts = [unicode(clean(d)) for d in texts]
tokens, vocab = preprocess.tokenize(texts, max_length, merge=False,
                                    n_threads=4)
corpus = Corpus()
# Make a ranked list of rare vs frequent words
corpus.update_word_count(tokens)
corpus.finalize()
# The tokenization uses spaCy indices, and so may have gaps
# between indices for words that aren't present in our dataset.
# This builds a new compact index
compact = corpus.to_compact(tokens)
# Remove extremely rare words
pruned = corpus.filter_count(compact, min_count=30)
# Convert the compactified arrays into bag of words arrays
bow = corpus.compact_to_bow(pruned)
# Words tend to have power law frequency, so selectively
# downsample the most prevalent words
clean = corpus.subsample_frequent(pruned)
# Now flatten a 2D array of document per row and word position
# per column to a 1D array of words. This will also remove skips
# and OoV words
doc_ids = np.arange(pruned.shape[0])
flattened, (doc_ids,) = corpus.compact_to_flat(pruned, doc_ids)
assert flattened.min() >= 0
# Fill in the pretrained word vectors
n_dim = 300
fn_wordvc = 'data/GoogleNews-vectors-negative300.bin'
vectors, s, f = corpus.compact_word_vectors(vocab, filename=fn_wordvc)
# Save all of the preprocessed files
pickle.dump(vocab, open('vocab.pkl', 'w'))
pickle.dump(corpus, open('corpus.pkl', 'w'))
np.save("flattened", flattened)
np.save("doc_ids", doc_ids)
np.save("pruned", pruned)
np.save("bow", bow)
np.save("vectors", vectors)

_Note_: the above printing mappings are from `corpus.py`:

    print compact, word, ' --> ', choice

Run [`lda2vec_run.py`](https://github.com/cemoody/lda2vec/raw/master/examples/twenty_newsgroups/lda2vec/lda2vec_run.py) in `examples/twenty_newsgroups/lda2vec` directory to generate `topics.pyldavis.npz` that contains the topic-to-word probabilities and frequencies. What's left is to visualize and label each topic from the it's prevalent words.

_Note_: make sure you've downloaded `lda2vec_model.py` beforehand

In [ ]:
!python lda2vec/lda2vec_run.py

^ `lda2vec_run.py` doesn't seem to be working right... so let's download the expected output of this script from GitHub and move on: [`topics.pyldavis.npz`](https://github.com/cemoody/lda2vec/blob/master/examples/twenty_newsgroups/lda2vec/topics.pyldavis.npz).

### Reading in the saved model topics

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
import seaborn

Install `pyLDAvis` with `pip install git+https://github.com/bmabey/pyLDAvis.git@master#egg=pyLDAvis`

In [ ]:
import pyLDAvis
pyLDAvis.enable_notebook()

In [ ]:
npz = np.load(open('topics.pyldavis.npz', 'r'))
dat = {k: v for (k, v) in npz.iteritems()}

In [ ]:
dat['vocab'] = dat['vocab'].tolist()
#dat['term_frequency'] = dat['term_frequency'] * 1.0 / dat['term_frequency'].sum()

In [ ]:
dat

In [ ]:
top_n = 10
topic_to_topwords = {}
for j, topic_to_word in enumerate(dat['topic_term_dists']):
    top = np.argsort(topic_to_word)[::-1][:top_n]
    msg = 'Topic %i '  % j
    top_words = [dat['vocab'][i].strip()[:35] for i in top]
    msg += ' '.join(top_words)
    print msg
    topic_to_topwords[j] = top_words

### Individual document topics

In [ ]:
from sklearn.datasets import fetch_20newsgroups
remove=('headers', 'footers', 'quotes')
texts = fetch_20newsgroups(subset='train', remove=remove)

In [ ]:
print texts.data[51]

In [ ]:
print texts.target_names[texts.target[51]]

In [ ]:
msg = "{weight:02d}% in topic {topic_id:02d} which has top words {text:s}"
for topic_id, weight in enumerate(dat['doc_topic_dists'][51]):
    if weight > 0.01:
        text = ', '.join(topic_to_topwords[topic_id])
        print msg.format(topic_id=topic_id, weight=int(weight * 100.0), text=text)